In [ ]:
# todo NOTE: this script does not reproduce original result since we removed device IDs 
# todo NOTE: also this script will encounter runtime/space error since we used device IDs as a key while merge dataframes 

import pandas as pd
import numpy as np

file_cleaned_flow = '../../Endpoint Mapping Data/Cleaned Flow/cleaned_flow_stat.csv'
cleaned_flow = pd.read_csv(file_cleaned_flow)

cleaned_flow['super_vendor'] = cleaned_flow.apply(
    lambda row: row.vendor_name.lower() if row.vendor_name==row.vendor_name
    else row.gpt_clean_vendor,
    axis=1)


cleaned_flow['generic_category'] = cleaned_flow.apply(
    lambda row: row.man_generic_category if row.man_generic_category==row.man_generic_category
    else row.gpt_generic_category,
    axis=1)


# read all party mapping files
file_all_party_mapping = '../../Endpoint Mapping Data/Domain Data/all_party_mapping.csv'
all_party_mapping = pd.read_csv(file_all_party_mapping)
# drop extra columns 
all_party_mapping = all_party_mapping[['super_vendor', 'domain', 'party_labels']].drop_duplicates()

# marge with clean flow
clean_flow_party_label = pd.merge(cleaned_flow,
                                  all_party_mapping,
                                  on=['super_vendor', 'domain'],
                                  how='left'
                                  )


# read user information file 
file_user_device_timezone = '../../Inspector Dataset/New data/user_device_timezone.csv'
user_device_timezone = pd.read_csv(file_user_device_timezone)
user_device_timezone = user_device_timezone[['device_id', 'user_key', 'user_country', 'timezone']]

# merge with timezone file 
clean_flow_party_label = pd.merge(clean_flow_party_label,
                                  user_device_timezone,
                                  on=['device_id'],
                                  how='left'
                                  )

print(len(clean_flow_party_label['device_id'].unique()))
unique_categories = clean_flow_party_label['generic_category'].unique()
print(list(unique_categories))

### RQ 1
#### How does the distribution of different types of contacted endpoints vary across various categories of IoT devices?

In [ ]:
unique_categories = ['Media/TV',
                     'Voice Assistant',
                     'Surveillance',
                     'Home Automation',
                     'Home Appliance ',
                     'Game Console',
                     'Generic IoT',
                     'Work Appliance',
                     'Vehicle']

devices_df = clean_flow_party_label
devices_df['average_out_byte_per_sec'] = devices_df['total_out_byte']/devices_df['flow_duration']
threshold = 1e6
devices_df=devices_df[devices_df['average_out_byte_per_sec'] < threshold]

In [ ]:
#original result
# distribution of different types of contacted endpoints vary across various categories of IoT devices
aggregated_data = []

for category in unique_categories:
    df = devices_df[(devices_df['generic_category']==category)]

    first_party = len(df[df['party_labels']==1]['domain'].unique())
    support_party = len(df[df['party_labels']==2]['domain'].unique())
    third_party = len(df[df['party_labels']==3]['domain'].unique())

    first = df[df['party_labels']==1].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "1"})
    support = df[df['party_labels']==2].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "2"})
    Third = df[df['party_labels']==3].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "3"})

    devices = df[['device_id']].drop_duplicates()
    devices = devices.merge(first,how ='left').merge(support,how ='left').merge(Third,how='left').fillna(0)

    first_party_mean, support_party_mean, third_party_mean = devices[['1','2','3']].mean()
    first_party_std, support_party_std, third_party_std = devices[['1','2','3']].std()


    first_party_up_stream = df[df['party_labels']==1]['average_out_byte_per_sec'].mean()
    support_party_up_stream = df[df['party_labels']==2]['average_out_byte_per_sec'].mean()
    third_party_up_stream = df[df['party_labels']==3]['average_out_byte_per_sec'].mean()


    data = {'Category': category,
            'first_party': first_party,
            'support_party': support_party,
            'third_party': third_party,
            'first_party_mean': first_party_mean,
            'support_party_mean': support_party_mean,
            'third_party_mean': third_party_mean,
            'first_party_std': first_party_std,
            'support_party_std': support_party_std,
            'third_party_std': third_party_std,
            'first_party_up_stream': first_party_up_stream,
            'support_party_up_stream': support_party_up_stream,
            'third_party_up_stream': third_party_up_stream
            }

    aggregated_data.append(data)
    # print(category.ljust(15), ' & ',
    #       str(first_party).rjust(6), ' & ','%.2f'.rjust(6) % first_party_mean, ' & ','%.2f'.rjust(6) % first_party_std, ' & ',
    #       str(support_party).rjust(6), ' & ', '%.2f'.rjust(6) % support_party_mean, ' & ','%.2f'.rjust(6) % support_party_std, ' & ',
    #       str(third_party).rjust(6), ' & ', '%.2f'.rjust(6) % third_party_mean, ' & ', '%.2f'.rjust(6) % third_party_std, '\\\\[3pt]')

rq_1_df = pd.DataFrame(aggregated_data)

### Block list

In [ ]:
import json
# Disconnect List

file_disconnect = '../../Domain Block Lists/Disconnect-List.json'

with open(file_disconnect, encoding="utf8") as f:
    disconnect = json.load(f)

  
disconnect = disconnect['categories']
print(disconnect.keys())


disconnect_ad_list = []
top_list = disconnect['Advertising']

for dc_item in top_list:
    for dc_item_key in dc_item.keys():
        inner_list = dc_item[dc_item_key]
        for inner_list_keys in inner_list.keys():
            disconnect_ad_list.extend(inner_list[inner_list_keys])     
            
disconnect_tracklist = []
top_list = disconnect['Analytics']

for dc_item in top_list:
    for dc_item_key in dc_item.keys():
        inner_list = dc_item[dc_item_key]
        for inner_list_keys in inner_list.keys():
            disconnect_tracklist.extend(inner_list[inner_list_keys])  
            
disconnect_list = []

for key in disconnect.keys():
    top_list = disconnect[key]
    for dc_item in top_list:
        for dc_item_key in dc_item.keys():
            inner_list = dc_item[dc_item_key]
            for inner_list_keys in inner_list.keys():
                disconnect_list.extend(inner_list[inner_list_keys])    

print(len(disconnect_list))

In [ ]:
# Create Database for WhoTracksMe
import sqlite3

#Connect to Database
con = sqlite3.connect('WhoTracksMe.db')
cur = con.cursor()
whotracksme_file = '../../Domain Block Lists/trackerdb.sql'
#read file
f= open(whotracksme_file,mode='r',encoding='utf8', newline='\n')
whotracksme_lines = [line.replace('\n', '') for line in f]
whotracksme_query = []
query = ''
for line in whotracksme_lines:
    query += line
    if query[-1] == ';':
        whotracksme_query.append(query)
        query = ''
for query in whotracksme_query:
    cur.execute(query)
con.commit()

# read a table       
sql_query = pd.read_sql('SELECT * FROM tracker_domains', con)
whotracksme_tracklist = sql_query['domain'].unique()

In [ ]:
## Easy Privacy 
file_easy_list = '../../Domain Block Lists/easylist.txt'
file_easy_privacy = '../../Domain Block Lists/easyprivacy.txt'


with open(file_easy_list, encoding="utf8") as f:
    easy_list = [line[2:].split("^")[0] for line in f if line.startswith('||')]
    
with open(file_easy_privacy, encoding="utf8") as f:
    easy_privacy = [line[2:].split("^")[0] for line in f if line.startswith('||')]

In [ ]:
file_duckduckgo = '../../Domain Block Lists/duckduckgo_categorized_trackers.csv'
duckduckgo_domains = pd.read_csv(file_duckduckgo)

duckduckgo_advertising = duckduckgo_domains[duckduckgo_domains['Advertising'] == 1]['domain'].unique()
duckduckgo_tracking = duckduckgo_domains[duckduckgo_domains['Ad Motivated Tracking'] == 1]['domain'].unique()

In [ ]:
category =   'Home Automation'

third_parties = devices_df[(devices_df['generic_category']==category) & (devices_df['party_labels']==3)]['domain'].unique()

ad_domain = []
track_domain = []
for party in third_parties:
    if (party in disconnect_list) \
    or (party in easy_list) \
    or (party in duckduckgo_advertising) : ad_domain.append(party)
        
    if (party in whotracksme_tracklist) \
    or (party in easy_privacy) \
    or (party in duckduckgo_tracking): track_domain.append(party)

print(len(ad_domain))
print(len(track_domain))
print(len(third_parties))

In [ ]:
### Find ad services in simple devices 
gpt_device_category_merged = pd.read_csv('../../Device Identification/Device Category Mapping/gpt_device_id_generic_category.csv')[['device_id', 'gpt_clean_vendor', 'gpt_clean_type','generic_category']].rename(columns={'generic_category':'gpt_generic_category'})

simple_devices = gpt_device_category_merged[(gpt_device_category_merged.gpt_clean_type.str.contains('Plug')) |
                                            (gpt_device_category_merged.gpt_clean_type.str.contains('Bulb')) |
                                            (gpt_device_category_merged.gpt_clean_type.str.contains('Switch')) |
                                            (gpt_device_category_merged.gpt_clean_type.str.contains('Light'))  ][['device_id']]
simple_devices = pd.merge(simple_devices, devices_df, on='device_id', how='inner')
simple_devices = simple_devices[simple_devices.generic_category=='Home Automation']
third_parties = simple_devices['domain'].unique()

ad_domain = []
track_domain = []
for party in third_parties:
    if (party in disconnect_list) \
            or (party in easy_list) \
            or (party in duckduckgo_advertising) : ad_domain.append(party)

    if (party in whotracksme_tracklist) \
            or (party in easy_privacy) \
            or (party in duckduckgo_tracking): track_domain.append(party)

print(len(ad_domain))
print(len(track_domain))
print(len(third_parties))

In [ ]:
ddict = {}
ad_domain.extend(track_domain)
for item in ad_domain:
    ddict[item]=1

print(len(ddict))

In [ ]:
## find advertiser percentage
third_parties = devices_df[(devices_df['party_labels']==3)]['domain'].unique()

ad_domain = []
track_domain = []
for party in third_parties:
    if (party in disconnect_list) \
            or (party in easy_list) \
            or (party in duckduckgo_advertising) : ad_domain.append(party)

    if (party in whotracksme_tracklist) \
            or (party in easy_privacy) \
            or (party in duckduckgo_tracking): track_domain.append(party)

print(len(ad_domain))
print(len(track_domain))
print(len(third_parties))

ddict = {}
ad_domain.extend(track_domain)
for item in ad_domain:
    ddict[item]=1

print(len(ddict))
print('percentage: ', 100*len(ddict)/len(third_parties))